In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import nltk
import string
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models

from gensim import corpora, models
from gensim.utils import effective_n_jobs
import collections
import re
import gc
import os

In [4]:
# get dataframe composed of all csvs in CNN folder
csvs = [x for x in os.listdir('../data/CNN/') if x.endswith('.csv')]
fns = [os.path.splitext(os.path.basename(x))[0] for x in csvs]
topics=[]
for i in fns:
    topics.append(i.split('_')[1])
d = {}
for i in range(len(fns)):
    d[topics[i]] = pd.read_csv(os.path.join('./CNN',csvs[i]))

In [6]:
# functions to lemmatize news texts
STOP = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['``', "''", "’", "“", "”","–", "\'s"])

def get_lemmas(text):
    '''
    Gets lemmas for a string input, excluding stop words, punctuation, as well
    as a set of study-specific stop-words
    '''
    lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(t)
              for t in nltk.word_tokenize((str(text).lower())) if t not in STOP
              ]
    return lemmas

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=2):
    '''
    Computes Coherence values for LDA models with differing numbers of topics.

    Returns list of models along with their respective coherence values (pick
    models with the highest coherence)
    '''
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = models.ldamulticore.LdaMulticore(corpus=corpus,
                                                 id2word=dictionary,
                                                 num_topics=num_topics,
                                                 workers=effective_n_jobs(-1))
        model_list.append(model)
        coherence_model = models.coherencemodel.CoherenceModel(model=model,
                                                          corpus=corpus,
                                                          dictionary=dictionary,
                                                          coherence='u_mass')
        coherence_values.append(coherence_model.get_coherence())
        
    return model_list, coherence_values

In [7]:
def train_lda(topic_df, model_name, num_topics, workers):
    '''
    Trains LDA model on a dataframe of news articles, saves model to disk
    Inputs:
        topic_df: dataframe of news articles
        model_name: name of model to be saved
        num_topics: number of topics to be trained
        workers: number of workers to be used in training
    '''
    # Get lemmas for each article
    lemmas = topic_df['text'].apply(get_lemmas)
    #reduce memory load
    del topic_df
    gc.collect()
    # Initialize Series of lemmas as Gensim Dictionary for further processing
    dictionary = corpora.Dictionary(lemmas)
    # Convert dictionary into bag of words format: list of (token_id, token_count) tuples
    bow_corpus = [dictionary.doc2bow(text) for text in lemmas]
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary,
                                                              corpus=bow_corpus,
                                                              texts=lemmas,
                                                              start=2,
                                                              limit=40,
                                                              step=6)
    # train LDA model
    ldamodel = models.ldamulticore.LdaMulticore(bow_corpus, num_topics=num_topics, id2word=dictionary, workers=workers, passes=20, iterations=400)
    ldamodel.save('{}.model'.format(model_name))

In [9]:
# train all topic models for CNN, commented out because it takes a long time to run

# for i in topics:
#     train_lda(d[i], i+'CNN', 5, 10)
#     print(i+'done')

In [25]:
# get pyLDAvis visualizations for all topic models under CNN
cnn_topics=[]
ps = []
for i in fns:
    cnn_topics.append(i.split('_')[1])
for i in cnn_topics:
    model = models.ldamodel.LdaModel.load('{}CNN.model'.format(i))
    topics = model.print_topics(num_words=20)
    for topic in topics:
        print(topic)
    lemmas = d[i]['text'].apply(get_lemmas)
    dictionary = corpora.Dictionary.load('{}CNN.model.id2word'.format(i))
    corpus = [dictionary.doc2bow(text) for text in lemmas]
    pyLDAvis.enable_notebook()
    p = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    ps.append(p)
    print(i+'done')
#     else:
#         model = models.ldamodel.LdaModel.load('{}cnn.model'.format(i))
#         topics = ldamodel.print_topics(num_words=20)
#         for topic in topics:
#             print(topic)
#         dictionary = corpora.Dictionary.load('{}cnn.id2word'.format(i))
#         p = pyLDAvis.gensim_models.prepare(model, bow_corpus, dictionary)
#         print(p)

(0, '0.019*"’" + 0.017*"“" + 0.017*"”" + 0.016*"heart" + 0.010*"said" + 0.010*"attack" + 0.007*"health" + 0.006*"patient" + 0.006*"study" + 0.006*"risk" + 0.005*"–" + 0.005*"people" + 0.005*"doctor" + 0.004*"hospital" + 0.004*"medical" + 0.004*"year" + 0.004*"disease" + 0.004*"one" + 0.003*"woman" + 0.003*"dr."')
(1, '0.022*"“" + 0.022*"’" + 0.022*"”" + 0.020*"said" + 0.013*"attack" + 0.012*"police" + 0.006*"people" + 0.006*"cnn" + 0.005*"one" + 0.004*"told" + 0.004*"two" + 0.004*"–" + 0.004*"according" + 0.004*"officer" + 0.003*"man" + 0.003*"say" + 0.003*"new" + 0.003*"suspect" + 0.003*"—" + 0.003*"victim"')
(2, '0.016*"’" + 0.012*"attack" + 0.011*"said" + 0.009*"”" + 0.009*"“" + 0.007*"ad" + 0.007*"video" + 0.006*"\'s" + 0.005*"security" + 0.005*"company" + 0.005*"content" + 0.005*"cnn" + 0.004*"government" + 0.004*"u" + 0.003*"system" + 0.003*"also" + 0.003*"new" + 0.003*"year" + 0.003*"one" + 0.003*"–"')
(3, '0.035*"’" + 0.026*"“" + 0.025*"”" + 0.015*"trump" + 0.010*"said" + 0.008

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


attackdone
(0, '0.034*"’" + 0.010*"cnn" + 0.009*"video" + 0.009*"source" + 0.008*"ad" + 0.008*"feedback" + 0.008*"”" + 0.008*"“" + 0.007*"u" + 0.007*"today" + 0.006*"trump" + 0.006*"new" + 0.005*"get" + 0.005*"said" + 0.004*"thing" + 0.004*"day" + 0.004*"–" + 0.004*"president" + 0.004*"people" + 0.004*"5"')
(1, '0.013*"’" + 0.006*"u" + 0.006*"“" + 0.006*"”" + 0.006*"first" + 0.004*"president" + 0.004*"nichols" + 0.004*"said" + 0.004*"new" + 0.003*"january" + 0.003*"state" + 0.003*"february" + 0.003*"may" + 0.003*"court" + 0.003*"cnn" + 0.003*"obama" + 0.003*"official" + 0.003*"announces" + 0.003*"woman" + 0.003*"biden"')
(2, '0.028*"’" + 0.024*"“" + 0.024*"”" + 0.018*"said" + 0.006*"people" + 0.006*"state" + 0.005*"cnn" + 0.005*"police" + 0.005*"–" + 0.004*"say" + 0.004*"year" + 0.004*"told" + 0.004*"family" + 0.004*"u" + 0.003*"one" + 0.003*"new" + 0.003*"biden" + 0.003*"school" + 0.003*"law" + 0.003*"gun"')
(3, '0.016*"’" + 0.014*"said" + 0.013*"“" + 0.013*"”" + 0.007*"fire" + 0.007*

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


Bidendone
(0, '0.031*"’" + 0.028*"“" + 0.028*"”" + 0.019*"said" + 0.013*"police" + 0.009*"black" + 0.007*"officer" + 0.006*"cnn" + 0.005*"people" + 0.005*"say" + 0.004*"one" + 0.004*"told" + 0.004*"–" + 0.003*"white" + 0.003*"family" + 0.003*"city" + 0.003*"video" + 0.003*"according" + 0.003*"death" + 0.003*"department"')
(1, '0.034*"’" + 0.023*"“" + 0.023*"”" + 0.015*"black" + 0.008*"said" + 0.007*"–" + 0.007*"people" + 0.007*"white" + 0.005*"american" + 0.005*"trump" + 0.004*"cnn" + 0.004*"president" + 0.004*"woman" + 0.004*"state" + 0.004*"right" + 0.004*"u" + 0.004*"one" + 0.003*"say" + 0.003*"would" + 0.003*"year"')
(2, '0.025*"“" + 0.024*"’" + 0.024*"”" + 0.012*"black" + 0.007*"\'s" + 0.005*"said" + 0.005*"–" + 0.004*"one" + 0.004*"year" + 0.004*"image" + 0.004*"first" + 0.004*"like" + 0.003*"cnn" + 0.003*"time" + 0.003*"new" + 0.003*"people" + 0.003*"film" + 0.003*"world" + 0.003*"player" + 0.003*"—"')
(3, '0.025*"’" + 0.011*"“" + 0.011*"”" + 0.011*"voter" + 0.010*"said" + 0.008

...

USdone
(0, '0.028*"’" + 0.022*"“" + 0.021*"”" + 0.011*"said" + 0.011*"gun" + 0.008*"police" + 0.007*"violence" + 0.006*"–" + 0.006*"people" + 0.005*"trump" + 0.005*"shooting" + 0.005*"cnn" + 0.004*"law" + 0.004*"one" + 0.004*"white" + 0.003*"officer" + 0.003*"state" + 0.003*"new" + 0.003*"city" + 0.003*"president"')
(1, '0.018*"’" + 0.015*"said" + 0.011*"“" + 0.011*"”" + 0.009*"people" + 0.008*"violence" + 0.006*"cnn" + 0.006*"attack" + 0.006*"killed" + 0.005*"police" + 0.005*"–" + 0.005*"country" + 0.005*"child" + 0.004*"year" + 0.004*"one" + 0.004*"state" + 0.004*"two" + 0.004*"group" + 0.004*"according" + 0.003*"say"')
(2, '0.028*"’" + 0.025*"“" + 0.024*"”" + 0.015*"said" + 0.007*"violence" + 0.005*"cnn" + 0.005*"–" + 0.004*"people" + 0.004*"one" + 0.004*"woman" + 0.004*"say" + 0.003*"year" + 0.003*"police" + 0.003*"also" + 0.003*"told" + 0.003*"time" + 0.003*"—" + 0.003*"israeli" + 0.003*"would" + 0.003*"new"')
(3, '0.023*"“" + 0.023*"”" + 0.023*"’" + 0.016*"said" + 0.007*"presiden

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


violencedone
(0, '0.018*"’" + 0.014*"“" + 0.014*"”" + 0.009*"said" + 0.008*"war" + 0.006*"people" + 0.005*"cnn" + 0.004*"child" + 0.004*"–" + 0.004*"image" + 0.004*"say" + 0.004*"one" + 0.004*"year" + 0.003*"crime" + 0.003*"country" + 0.003*"government" + 0.003*"world" + 0.003*"refugee" + 0.003*"police" + 0.003*"city"')
(1, '0.022*"’" + 0.018*"“" + 0.017*"”" + 0.010*"war" + 0.006*"said" + 0.006*"–" + 0.005*"one" + 0.005*"year" + 0.005*"\'s" + 0.004*"world" + 0.004*"time" + 0.004*"cnn" + 0.003*"first" + 0.003*"people" + 0.003*"day" + 0.003*"life" + 0.003*"say" + 0.003*"family" + 0.003*"new" + 0.003*"story"')
(2, '0.021*"’" + 0.017*"“" + 0.017*"”" + 0.013*"said" + 0.011*"syria" + 0.008*"war" + 0.008*"north" + 0.008*"syrian" + 0.007*"korea" + 0.007*"isi" + 0.006*"force" + 0.006*"military" + 0.006*"–" + 0.005*"attack" + 0.005*"state" + 0.005*"group" + 0.005*"country" + 0.005*"government" + 0.005*"korean" + 0.005*"u"')
(3, '0.029*"’" + 0.020*"”" + 0.020*"“" + 0.011*"trump" + 0.010*"said" + 

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


wardone


In [44]:
pyLDAvis.enable_notebook
ps[0]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.098525 -0.061884       1        1  34.613375
1     -0.050958 -0.049368       2        1  29.912925
3     -0.033107  0.123029       3        1  21.312719
2     -0.004394  0.012214       4        1  10.994340
0      0.186984 -0.023992       5        1   3.166641, topic_info=         Term          Freq         Total Category  logprob  loglift
12911       ’  83802.000000  83802.000000  Default  30.0000  30.0000
12912       “  72606.000000  72606.000000  Default  29.0000  29.0000
12913       ”  71969.000000  71969.000000  Default  28.0000  28.0000
6797    trump  13257.000000  13257.000000  Default  27.0000  27.0000
15     attack  46805.000000  46805.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
147       say    378.760225  11012.625650   Topic5  -5.7437   0.0826
11       also    363.468698  11414.870189   Topic5  -5.7849   0.0055
1091      cnn    377.586705  21237.386944   Topic5  -5.7468  -0.5772
120       new    328.385256   9823.609767   Topic5  -5.8864   0.0541
637      time    323.210485   8557.931678   Topic5  -5.9023   0.1762

[485 rows x 6 columns], token_table=       Topic      Freq Term
term                       
2          1  0.200258   's
2          2  0.196952   's
2          3  0.158501   's
2          4  0.417218   's
2          5  0.027142   's
...      ...       ...  ...
12913      1  0.304421    ”
12913      2  0.333864    ”
12913      3  0.281814    ”
12913      4  0.052550    ”
12913      5  0.027345    ”

[1329 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 3, 1])

In [45]:
ps[1]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.093625 -0.043691       1        1  40.102411
2     -0.114901  0.024303       2        1  36.380054
3      0.008403  0.143195       3        1  15.100211
1     -0.024499 -0.101591       4        1   6.892989
4      0.224622 -0.022217       5        1   1.524336, topic_info=           Term          Freq         Total Category  logprob  loglift
779        fire   1798.000000   1798.000000  Default  30.0000  30.0000
2961      image   2269.000000   2269.000000  Default  29.0000  29.0000
256        said  12401.000000  12401.000000  Default  28.0000  28.0000
10164  feedback   3560.000000   3560.000000  Default  27.0000  27.0000
330           “  16163.000000  16163.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
3905        gas     41.975291   1045.808008   Topic5  -6.0254   0.9681
2475       near     37.026551    831.125816   Topic5  -6.1508   1.0725
324        year     38.703477   3640.869939   Topic5  -6.1065  -0.3604
5803     oregon     31.784918    231.572938   Topic5  -6.3034   2.1977
4719        san     32.121776    404.138253   Topic5  -6.2929   1.6514

[455 rows x 6 columns], token_table=      Topic      Freq Term
term                      
762       1  0.423289   's
762       2  0.235833   's
762       3  0.272978   's
762       4  0.060902   's
762       5  0.006479   's
...     ...       ...  ...
331       1  0.217292    ”
331       2  0.611818    ”
331       3  0.138361    ”
331       4  0.031024    ”
331       5  0.001495    ”

[1124 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2, 5])

In [46]:
ps[2]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.050991 -0.004194       1        1  35.117783
0     -0.082326 -0.023041       2        1  24.155585
2     -0.050397 -0.043660       3        1  20.085796
4      0.138294 -0.081089       4        1  15.210223
3      0.045420  0.151984       5        1   5.430614, topic_info=         Term           Freq          Total Category  logprob  loglift
16176       ’  119508.000000  119508.000000  Default  30.0000  30.0000
2274   police   15695.000000   15695.000000  Default  29.0000  29.0000
5097    voter    5347.000000    5347.000000  Default  28.0000  28.0000
154      said   44441.000000   44441.000000  Default  27.0000  27.0000
16177       “   91086.000000   91086.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
511     first     832.281010   10269.962687   Topic5  -5.6362   0.4003
361       one     704.719998   16426.077598   Topic5  -5.8026  -0.2357
127       new     665.837761   12577.902395   Topic5  -5.8593  -0.0255
5325        —     613.446610   10574.862746   Topic5  -5.9413   0.0660
26       also     601.900444   12646.339137   Topic5  -5.9603  -0.1319

[510 rows x 6 columns], token_table=       Topic      Freq Term
term                       
0          1  0.036267   's
0          2  0.077835   's
0          3  0.523923   's
0          4  0.344446   's
0          5  0.017483   's
...      ...       ...  ...
16178      1  0.383223    ”
16178      2  0.318603    ”
16178      3  0.228668    ”
16178      4  0.040637    ”
16178      5  0.028868    ”

[1445 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 5, 4])

In [47]:
ps[3]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.027071  0.087872       1        1  45.653866
3      0.072907  0.028756       2        1  24.156838
1      0.067256 -0.012047       3        1  15.070120
2      0.031366 -0.079711       4        1   8.868990
4     -0.144458 -0.024870       5        1   6.250186, topic_info=          Term          Freq         Total Category  logprob  loglift
359          “  10962.000000  10962.000000  Default  30.0000  30.0000
360          ”  10921.000000  10921.000000  Default  29.0000  29.0000
393         ad   3669.000000   3669.000000  Default  28.0000  28.0000
4314  feedback   3536.000000   3536.000000  Default  27.0000  27.0000
273       said   6777.000000   6777.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
340        way     89.673857   1017.653179   Topic5  -6.3503   0.3435
329          u    103.400276   3976.869885   Topic5  -6.2079  -0.8771
1532     north     88.217933    945.500271   Topic5  -6.3667   0.4007
169       know     91.587462   1579.460048   Topic5  -6.3292  -0.0750
355          –     93.870643   3206.166919   Topic5  -6.3046  -0.7584

[459 rows x 6 columns], token_table=      Topic      Freq Term
term                      
826       1  0.436487   's
826       2  0.118422   's
826       3  0.082461   's
826       4  0.077501   's
826       5  0.284585   's
...     ...       ...  ...
360       1  0.259864    ”
360       2  0.388697    ”
360       3  0.228457    ”
360       4  0.108963    ”
360       5  0.014010    ”

[1036 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3, 5])

In [48]:
ps[4]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.008429 -0.095021       1        1  26.849898
0      0.073146  0.066601       2        1  26.835663
2     -0.054116  0.082777       3        1  19.695298
4      0.079864 -0.028981       4        1  14.697282
1     -0.107323 -0.025376       5        1  11.921859, topic_info=         Term          Freq         Total Category  logprob  loglift
9215    trump  17955.000000  17955.000000  Default  30.0000  30.0000
4172  ukraine  10599.000000  10599.000000  Default  29.0000  29.0000
2884   russia  12845.000000  12845.000000  Default  28.0000  28.0000
3865  russian   9705.000000   9705.000000  Default  27.0000  27.0000
114     syria  12035.000000  12035.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
396      life   1117.086265   4447.202706   Topic5  -6.1883   0.7452
6613        —   1201.892326  10296.462771   Topic5  -6.1151  -0.0211
532     group   1186.176118  10156.793241   Topic5  -6.1283  -0.0206
403       new   1154.778791  11536.648121   Topic5  -6.1551  -0.1748
638       two   1118.144205   9629.624077   Topic5  -6.1874  -0.0264

[493 rows x 6 columns], token_table=       Topic      Freq Term
term                       
0          1  0.128789   's
0          2  0.395049   's
0          3  0.183054   's
0          4  0.094473   's
0          5  0.198558   's
...      ...       ...  ...
16401      1  0.265337    ”
16401      2  0.272448    ”
16401      3  0.231573    ”
16401      4  0.146491    ”
16401      5  0.084149    ”

[1416 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 5, 2])

In [49]:
ps[5]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.011811 -0.035010       1        1  24.493728
2     -0.089612 -0.014765       2        1  23.591055
4     -0.044006  0.079792       3        1  21.599344
1      0.119002  0.023115       4        1  17.175098
0      0.002805 -0.053132       5        1  13.140776, topic_info=            Term          Freq         Total Category  logprob  loglift
8135       trump  12895.000000  12895.000000  Default  30.0000  30.0000
748    president   8811.000000   8811.000000  Default  29.0000  29.0000
482       police  22958.000000  22958.000000  Default  28.0000  28.0000
13148          ’  98055.000000  98055.000000  Default  27.0000  27.0000
95           law  10585.000000  10585.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
13189          –   1720.046351  18012.596468   Topic5  -5.6321  -0.3193
930         city   1362.340009   8372.298226   Topic5  -5.8653   0.2137
6      according   1421.210878  10250.272122   Topic5  -5.8230   0.0537
128       people   1463.266340  13627.021506   Topic5  -5.7938  -0.2019
122          one   1430.078068  13016.795389   Topic5  -5.8168  -0.1791

[501 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
65178      3  0.991284  abimbola
13753      1  0.005527       abu
13753      2  0.978349       abu
13753      3  0.005527       abu
13753      4  0.005527       abu
...      ...       ...       ...
13150      1  0.239399         ”
13150      2  0.191786         ”
13150      3  0.272580         ”
13150      4  0.167425         ”
13150      5  0.128803         ”

[1338 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 2, 1])

In [50]:
ps[6]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.115893  0.031223       1        1  25.805466
2     -0.002506  0.054466       2        1  25.150630
3      0.040286 -0.024628       3        1  22.722743
4     -0.005382 -0.080783       4        1  16.632838
1     -0.148291  0.019722       5        1   9.688324, topic_info=            Term          Freq         Total Category  logprob  loglift
739      clinton  17157.000000  17157.000000  Default  30.0000  30.0000
176        voter  16972.000000  16972.000000  Default  29.0000  29.0000
125         poll   9457.000000   9457.000000  Default  28.0000  28.0000
1847       biden  20101.000000  20101.000000  Default  27.0000  27.0000
702   republican  30906.000000  30906.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1724      sander   1693.635371  15895.527287   Topic5  -5.5145   0.0951
277    president   1832.266014  24616.856228   Topic5  -5.4359  -0.2636
183         year   1603.474994  13801.662437   Topic5  -5.5692   0.1816
475         race   1485.253647  12504.300366   Topic5  -5.6458   0.2038
5909           –   1371.644530  32071.876969   Topic5  -5.7254  -0.8177

[516 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.150511   's
0         2  0.173976   's
0         3  0.486061   's
0         4  0.013073   's
0         5  0.176323   's
...     ...       ...  ...
5912      1  0.347533    ”
5912      2  0.191391    ”
5912      3  0.284476    ”
5912      4  0.168264    ”
5912      5  0.008338    ”

[1450 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 5, 2])

In [51]:
ps[7]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.021803 -0.029512       1        1  29.241565
1      0.025247  0.136242       2        1  24.877953
4      0.038695 -0.055294       3        1  22.706899
0      0.071436 -0.038236       4        1  15.560497
2     -0.157181 -0.013200       5        1   7.613086, topic_info=       Term           Freq          Total Category  logprob  loglift
5699      ’  104411.000000  104411.000000  Default  30.0000  30.0000
5700      “   71240.000000   71240.000000  Default  29.0000  29.0000
5701      ”   70069.000000   70069.000000  Default  28.0000  28.0000
8653  trump   19368.000000   19368.000000  Default  27.0000  27.0000
1        's   16921.000000   16921.000000  Default  26.0000  26.0000
...     ...            ...            ...      ...      ...      ...
233    said    1195.362041   40697.477719   Topic5  -5.5167  -0.9524
471    time     884.844926   11306.452506   Topic5  -5.8175   0.0276
741     two     787.800217    7128.920998   Topic5  -5.9337   0.3726
290   would     809.600184   13725.496042   Topic5  -5.9064  -0.2552
3150      —     795.701535   10504.540929   Topic5  -5.9237  -0.0050

[496 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.016602  're
0         2  0.908672  're
0         3  0.064194  're
0         4  0.004427  're
0         5  0.006641  're
...     ...       ...  ...
5701      1  0.374316    ”
5701      2  0.055103    ”
5701      3  0.306026    ”
5701      4  0.210792    ”
5701      5  0.053761    ”

[1391 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 1, 3])

In [52]:
ps[8]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.096055 -0.129909       1        1  30.447855
4     -0.060107 -0.030381       2        1  24.229528
3     -0.115050  0.080184       3        1  22.707226
0     -0.068824  0.079045       4        1  21.804935
2      0.340036  0.001061       5        1   0.810457, topic_info=          Term          Freq         Total Category  logprob  loglift
0           's   6788.000000   6788.000000  Default  30.0000  30.0000
70         gun  48179.000000  48179.000000  Default  29.0000  29.0000
120     police  25349.000000  25349.000000  Default  28.0000  28.0000
6046     image   3532.000000   3532.000000  Default  27.0000  27.0000
108        new  13068.000000  13068.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
868   facebook     73.320382   1578.016785   Topic5  -6.1222   1.7462
2154  military     64.013208   1558.056532   Topic5  -6.2579   1.6232
1914        al     56.483774    498.155402   Topic5  -6.3830   2.6384
717       july     61.482804   1542.403855   Topic5  -6.2982   1.5930
1817     world     59.237617   2513.086948   Topic5  -6.3354   1.0676

[495 rows x 6 columns], token_table=       Topic      Freq Term
term                       
0          1  0.230544   's
0          2  0.116819   's
0          3  0.253083   's
0          4  0.318784   's
0          5  0.080727   's
...      ...       ...  ...
15406      1  0.303175    ”
15406      2  0.347546    ”
15406      3  0.184845    ”
15406      4  0.164410    ”
15406      5  0.000036    ”

[1291 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 1, 3])

In [53]:
ps[9]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.052145  0.004116       1        1  23.352159
1      0.058149 -0.018531       2        1  21.132322
3     -0.101417 -0.062530       3        1  18.932800
0     -0.016806  0.106058       4        1  18.311787
4      0.112219 -0.029113       5        1  18.270932, topic_info=            Term          Freq         Total Category  logprob  loglift
672       border  17023.000000  17023.000000  Default  30.0000  30.0000
804        court  10297.000000  10297.000000  Default  29.0000  29.0000
388   republican  14665.000000  14665.000000  Default  28.0000  28.0000
4317           “  81803.000000  81803.000000  Default  27.0000  27.0000
4318           ”  81394.000000  81394.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
4315           –   2942.990836  24904.618398   Topic5  -5.6066  -0.4358
2154       trump   3194.785608  41921.462841   Topic5  -5.5245  -0.8744
11          also   2506.617682  13471.988014   Topic5  -5.7671   0.0182
223        would   2515.482015  18864.075609   Topic5  -5.7636  -0.3149
224         year   2472.555946  15445.510692   Topic5  -5.7808  -0.1322

[541 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2         1  0.104300   's
2         2  0.031853   's
2         3  0.778936   's
2         4  0.031350   's
2         5  0.053557   's
...     ...       ...  ...
4318      1  0.361460    ”
4318      2  0.227987    ”
4318      3  0.030886    ”
4318      4  0.190515    ”
4318      5  0.189152    ”

[1568 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 1, 5])

In [54]:
ps[10]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.084435  0.044621       1        1  25.386416
3      0.009854 -0.074399       2        1  20.755061
2      0.069074 -0.000947       3        1  18.972444
1     -0.084791  0.089226       4        1  17.689449
4     -0.078572 -0.058500       5        1  17.196630, topic_info=           Term          Freq         Total Category  logprob  loglift
1693     russia  19008.000000  19008.000000  Default  30.0000  30.0000
1718    ukraine  19523.000000  19523.000000  Default  29.0000  29.0000
794     russian  17212.000000  17212.000000  Default  28.0000  28.0000
2797      trump  10666.000000  10666.000000  Default  27.0000  27.0000
1719  ukrainian   8579.000000   8579.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
315       would   1624.600871  13139.730334   Topic5  -5.9443  -0.3299
148    invasion   1269.831239   8589.483266   Topic5  -6.1907  -0.1512
695       first   1218.296939   7151.356521   Topic5  -6.2321  -0.0094
268       state   1228.761930  12195.747620   Topic5  -6.2236  -0.5346
403         day   1169.515425   7556.106927   Topic5  -6.2730  -0.1053

[507 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1         1  0.110831   's
1         2  0.179465   's
1         3  0.244540   's
1         4  0.120491   's
1         5  0.344695   's
...     ...       ...  ...
8050      1  0.272667    ”
8050      2  0.183852    ”
8050      3  0.182771    ”
8050      4  0.129517    ”
8050      5  0.231208    ”

[1405 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2, 5])

In [55]:
ps[11]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.027467  0.058014       1        1  35.185177
1      0.070149  0.026930       2        1  32.881055
3      0.002953  0.056022       3        1  16.214650
2      0.068262 -0.115767       4        1   9.101957
4     -0.168831 -0.025200       5        1   6.617161, topic_info=         Term         Freq        Total Category  logprob  loglift
4769    trump  4611.000000  4611.000000  Default  30.0000  30.0000
1          's  3047.000000  3047.000000  Default  29.0000  29.0000
3901  clinton  1604.000000  1604.000000  Default  28.0000  28.0000
484   company  1103.000000  1103.000000  Default  27.0000  27.0000
1205    image  1998.000000  1998.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
114       gay   169.782194  4727.989211   Topic5  -6.1423  -0.6112
2272    march   141.182353   926.135675   Topic5  -6.3268   0.8345
109     first   152.413047  2974.186312   Topic5  -6.2502  -0.2556
712     world   145.401893  1813.624704   Topic5  -6.2973   0.1919
4769    trump   142.325923  4611.273564   Topic5  -6.3187  -0.7626

[466 rows x 6 columns], token_table=      Topic      Freq Term
term                      
420       1  0.219800   'm
420       2  0.027475   'm
420       3  0.109900   'm
420       4  0.054950   'm
420       5  0.583844   'm
...     ...       ...  ...
8759      1  0.466563    ”
8759      2  0.358252    ”
8759      3  0.119118    ”
8759      4  0.053259    ”
8759      5  0.002773    ”

[1111 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3, 5])

In [56]:
ps[12]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.035035  0.056118       1        1  35.361285
2     -0.024955 -0.023702       2        1  30.584632
1      0.138117  0.052008       3        1  16.461846
3      0.016556 -0.124077       4        1  11.680508
4     -0.094682  0.039653       5        1   5.911729, topic_info=            Term          Freq         Total Category  logprob  loglift
5853           ’  85058.000000  85058.000000  Default  30.0000  30.0000
693      officer  37316.000000  37316.000000  Default  29.0000  29.0000
119         said  72682.000000  72682.000000  Default  28.0000  28.0000
5854           “  73407.000000  73407.000000  Default  27.0000  27.0000
5855           ”  72840.000000  72840.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
4522      mother    482.168016   2878.989570   Topic5  -6.0725   1.0413
131         time    569.996960   9027.813451   Topic5  -5.9052   0.0658
120          say    593.133747  12224.805663   Topic5  -5.8654  -0.1976
647   department    570.708936  14107.477294   Topic5  -5.9039  -0.3793
3884       child    518.448641   5002.037304   Topic5  -5.9999   0.5615

[493 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1         1  0.246047   's
1         2  0.144018   's
1         3  0.316839   's
1         4  0.237731   's
1         5  0.055376   's
...     ...       ...  ...
5855      1  0.382357    ”
5855      2  0.387849    ”
5855      3  0.092943    ”
5855      4  0.087287    ”
5855      5  0.049547    ”

[1264 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5])

In [57]:
ps[13]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.076224  0.002923       1        1  31.089353
1     -0.009664 -0.033523       2        1  26.918289
0     -0.045384  0.086077       3        1  17.799550
3     -0.061382 -0.057051       4        1  15.941220
2      0.192655  0.001575       5        1   8.251587, topic_info=                Term          Freq         Total Category  logprob  loglift
155             hong  10771.000000  10771.000000  Default  30.0000  30.0000
169             kong  10367.000000  10367.000000  Default  29.0000  29.0000
1379       protester  19193.000000  19193.000000  Default  28.0000  28.0000
2318           image  12407.000000  12407.000000  Default  27.0000  27.0000
1734          police  24624.000000  24624.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
16507              “   2066.757555  86476.246572   Topic5  -5.1859  -1.2391
1320   demonstration   1127.990543   4613.916728   Topic5  -5.7914   1.0861
242             said   1484.913909  51741.974119   Topic5  -5.5165  -1.0561
1060         student   1111.115796   5830.905605   Topic5  -5.8065   0.8370
575              law   1119.584863   8115.683248   Topic5  -5.7989   0.5139

[510 rows x 6 columns], token_table=       Topic      Freq Term
term                       
1          1  0.512216   's
1          2  0.132877   's
1          3  0.055282   's
1          4  0.089676   's
1          5  0.209970   's
...      ...       ...  ...
16508      1  0.307988    ”
16508      2  0.282673    ”
16508      3  0.210408    ”
16508      4  0.174486    ”
16508      5  0.024436    ”

[1398 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 4, 3])

In [58]:
ps[14]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.012990  0.025221       1        1  33.370012
0      0.098576 -0.072878       2        1  29.209304
2     -0.002925  0.111802       3        1  18.236949
1      0.053513 -0.014921       4        1  17.240371
4     -0.162153 -0.049224       5        1   1.943364, topic_info=             Term           Freq          Total Category  logprob  loglift
4338        trump   28541.000000   28541.000000  Default  30.0000  30.0000
3570        video    4111.000000    4111.000000  Default  29.0000  29.0000
205          said   50262.000000   50262.000000  Default  28.0000  28.0000
808    republican    9034.000000    9034.000000  Default  27.0000  27.0000
17238           ’  114696.000000  114696.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
207           say     185.252297   11954.476150   Topic5  -6.0723  -0.2264
152         never     146.533271    3183.273612   Topic5  -6.3068   0.8623
25           also     157.195955   11511.727420   Topic5  -6.2366  -0.3529
265     according     151.885540    7802.208285   Topic5  -6.2709   0.0017
965        people     152.171093   12304.374481   Topic5  -6.2691  -0.4520

[512 rows x 6 columns], token_table=       Topic      Freq Term
term                       
1          1  0.250673   's
1          2  0.063783   's
1          3  0.219005   's
1          4  0.441243   's
1          5  0.025201   's
...      ...       ...  ...
17240      1  0.405975    ”
17240      2  0.306212    ”
17240      3  0.136390    ”
17240      4  0.144804    ”
17240      5  0.006624    ”

[1401 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2, 5])

In [59]:
ps[15]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.022038  0.052756       1        1  30.931202
3      0.094813  0.035113       2        1  23.340606
4      0.089776 -0.009829       3        1  19.602513
2     -0.043482 -0.136561       4        1  16.429153
0     -0.163145  0.058521       5        1   9.696525, topic_info=          Term           Freq          Total Category  logprob  loglift
1670      iran    8173.000000    8173.000000  Default  30.0000  30.0000
11793        ’  110425.000000  110425.000000  Default  29.0000  29.0000
6475     trump   14162.000000   14162.000000  Default  28.0000  28.0000
2939   russian    7054.000000    7054.000000  Default  27.0000  27.0000
1961     korea    6103.000000    6103.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
21        also    1184.453892   13478.227222   Topic5  -5.9034  -0.0984
158       last    1051.396571    7831.868977   Topic5  -6.0226   0.3253
252       said    1315.472264   43123.541798   Topic5  -5.7985  -1.1565
1469       cnn    1062.637505   18148.070387   Topic5  -6.0119  -0.5044
197        new    1008.091972   13026.082665   Topic5  -6.0646  -0.2255

[486 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.321531   's
0         2  0.133228   's
0         3  0.161250   's
0         4  0.154851   's
0         5  0.229125   's
...     ...       ...  ...
5570      1  0.053278    •
5570      2  0.036364    •
5570      3  0.082031    •
5570      4  0.823690    •
5570      5  0.004228    •

[1356 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 3, 1])

In [60]:
ps[16]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.071695 -0.003017       1        1  37.518623
0     -0.036271 -0.098052       2        1  26.487355
2     -0.055994  0.014470       3        1  21.371182
4     -0.006917  0.099893       4        1  12.317845
3      0.170878 -0.013294       5        1   2.304996, topic_info=              Term          Freq         Total Category  logprob  loglift
412              ’  22590.000000  22590.000000  Default  30.0000  30.0000
413              “   8733.000000   8733.000000  Default  29.0000  29.0000
414              ”   8706.000000   8706.000000  Default  28.0000  28.0000
314           said   5503.000000   5503.000000  Default  27.0000  27.0000
6295            ad   3603.000000   3603.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
1519  intelligence     29.390490    533.327158   Topic5  -6.3497   0.8716
249            one     33.523975   2383.181062   Topic5  -6.2181  -0.4938
409           year     31.428218   2370.281933   Topic5  -6.2826  -0.5530
369           time     30.599879   1624.576182   Topic5  -6.3093  -0.2019
463            day     30.673822   2471.767710   Topic5  -6.3069  -0.6192

[461 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1227      1  0.328466   's
1227      2  0.110339   's
1227      3  0.221954   's
1227      4  0.336757   's
1227      5  0.002551   's
...     ...       ...  ...
415       1  0.156228    …
415       2  0.262746    …
415       3  0.536145    …
415       4  0.017753    …
415       5  0.026630    …

[1055 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 5, 4])

In [61]:
ps[17]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.038285  0.094449       1        1  31.469194
2     -0.042900  0.028774       2        1  24.606118
0      0.122888 -0.056901       3        1  16.545390
4     -0.065048 -0.091419       4        1  15.716958
1     -0.053225  0.025097       5        1  11.662340, topic_info=           Term          Freq         Total Category  logprob  loglift
1912     police  14473.000000  14473.000000  Default  30.0000  30.0000
32       attack  27805.000000  27805.000000  Default  29.0000  29.0000
4508      trump  16271.000000  16271.000000  Default  28.0000  28.0000
1366        isi  21808.000000  21808.000000  Default  27.0000  27.0000
139       force  11338.000000  11338.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
10    according   1265.408270   9335.677654   Topic5  -6.0631   0.1504
532         two   1286.103813  11317.441643   Topic5  -6.0469  -0.0259
258    security   1278.604494  11898.550374   Topic5  -6.0527  -0.0818
298        time   1235.882582  10543.489621   Topic5  -6.0867   0.0051
5166          —   1233.108002  10463.150907   Topic5  -6.0889   0.0105

[518 rows x 6 columns], token_table=       Topic      Freq Term
term                       
2          1  0.080558   's
2          2  0.125475   's
2          3  0.191325   's
2          4  0.143578   's
2          5  0.459247   's
...      ...       ...  ...
13031      1  0.422067    ”
13031      2  0.286610    ”
13031      3  0.106090    ”
13031      4  0.119603    ”
13031      5  0.065634    ”

[1461 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 5, 2])

In [62]:
ps[18]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.109763  0.007102       1        1  52.151751
3      0.070081  0.128805       2        1  16.184537
4      0.065147 -0.041331       3        1  15.731842
1      0.079121 -0.093661       4        1  15.556529
0     -0.324111 -0.000916       5        1   0.375340, topic_info=             Term           Freq          Total Category  logprob  loglift
2              's   15261.000000   15261.000000  Default  30.0000  30.0000
14206           ’  110608.000000  110608.000000  Default  29.0000  29.0000
14207           “   69331.000000   69331.000000  Default  28.0000  28.0000
14208           ”   69087.000000   69087.000000  Default  27.0000  27.0000
256         trump  117845.000000  117845.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
9463      fighter      24.112787     234.081374   Topic5  -6.3512   3.3122
2046            —      39.594618    9966.770325   Topic5  -5.8553   0.0568
3722         navy      23.199962     184.578638   Topic5  -6.3898   3.5112
815          u.s.      31.764782    3159.592794   Topic5  -6.0756   0.9853
1374   production      24.030360     282.390911   Topic5  -6.3547   3.1211

[464 rows x 6 columns], token_table=       Topic      Freq Term
term                       
0          1  0.080891  'll
0          2  0.896543  'll
0          3  0.006741  'll
0          4  0.016852  'll
1          1  0.082911  're
...      ...       ...  ...
14208      1  0.739583    ”
14208      2  0.003242    ”
14208      3  0.112423    ”
14208      4  0.144657    ”
14208      5  0.000101    ”

[1229 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 2, 1])

In [63]:
ps[19]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.132294  0.036939       1        1  25.704859
3      0.004422 -0.064131       2        1  23.370507
0      0.029290 -0.073408       3        1  23.032451
2     -0.134757 -0.006673       4        1  15.494105
4     -0.031250  0.107273       5        1  12.398078, topic_info=              Term          Freq         Total Category  logprob  loglift
11152        trump  28854.000000  28854.000000  Default  30.0000  30.0000
202        russian  29083.000000  29083.000000  Default  29.0000  29.0000
201         russia  33778.000000  33778.000000  Default  28.0000  28.0000
17762  impeachment   7519.000000   7519.000000  Default  27.0000  27.0000
880      ukrainian  17313.000000  17313.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
254        ukraine   1782.442970  42978.903328   Topic5  -5.7601  -1.0951
21            also   1405.399682  15308.460528   Topic5  -5.9978  -0.3005
245           time   1330.556052  11218.178034   Topic5  -6.0525  -0.0443
379            say   1303.006865  10048.240924   Topic5  -6.0734   0.0449
266          world   1285.713728   9398.592153   Topic5  -6.0868   0.0984

[494 rows x 6 columns], token_table=       Topic      Freq Term
term                       
3          1  0.024280   's
3          2  0.179079   's
3          3  0.061920   's
3          4  0.593375   's
3          5  0.141439   's
...      ...       ...  ...
13096      1  0.301909    ”
13096      2  0.293256    ”
13096      3  0.240443    ”
13096      4  0.084968    ”
13096      5  0.079427    ”

[1442 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])

In [64]:
ps[20]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.101705  0.015202       1        1  52.385253
4      0.055190  0.034199       2        1  20.884129
2      0.002224 -0.084406       3        1  12.093107
1     -0.075746 -0.042926       4        1   8.349742
3     -0.083373  0.077931       5        1   6.287769, topic_info=           Term          Freq         Total Category  logprob  loglift
362           “   4578.000000   4578.000000  Default  30.0000  30.0000
363           ”   4578.000000   4578.000000  Default  29.0000  29.0000
361           ’  12272.000000  12272.000000  Default  28.0000  28.0000
455        said   2644.000000   2644.000000  Default  27.0000  27.0000
4929         ad   2338.000000   2338.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
19         also     64.041477   1051.734171   Topic5  -6.0039  -0.0321
369   according     52.986310    595.792034   Topic5  -6.1934   0.3467
356       would     50.352189    819.147535   Topic5  -6.2444  -0.0227
1459      group     46.296457    404.476178   Topic5  -6.3284   0.5990
794           —     46.710556    709.917220   Topic5  -6.3195   0.0454

[437 rows x 6 columns], token_table=      Topic      Freq Term
term                      
474       1  0.505982   's
474       2  0.265282   's
474       3  0.077843   's
474       4  0.135202   's
474       5  0.014340   's
...     ...       ...  ...
363       1  0.364990    ”
363       2  0.188720    ”
363       3  0.188283    ”
363       4  0.129745    ”
363       5  0.127998    ”

[886 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 3, 2, 4])

In [65]:
ps[21]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.093450 -0.055616       1        1  25.867024
4     -0.098899 -0.062601       2        1  21.157511
0      0.009723 -0.023930       3        1  19.193027
3      0.017587  0.059481       4        1  18.470064
1     -0.021861  0.082666       5        1  15.312374, topic_info=          Term           Freq          Total Category  logprob  loglift
186          ’  101400.000000  101400.000000  Default  30.0000  30.0000
4889        ad    7459.000000    7459.000000  Default  29.0000  29.0000
187          “   66857.000000   66857.000000  Default  28.0000  28.0000
188          ”   66416.000000   66416.000000  Default  27.0000  27.0000
5768  feedback    6427.000000    6427.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
1079      home    1441.778135    6329.107141   Topic5  -5.9642   0.3972
576          ‘    1291.825956    4849.225284   Topic5  -6.0740   0.5537
185          —    1486.533120    8775.782664   Topic5  -5.9337   0.1010
326      first    1443.293362    8442.401254   Topic5  -5.9632   0.1102
17        also    1269.728420    9005.108919   Topic5  -6.0913  -0.0825

[526 rows x 6 columns], token_table=      Topic      Freq Term
term                      
711       1  0.204456   's
711       2  0.512640   's
711       3  0.064355   's
711       4  0.115919   's
711       5  0.102528   's
...     ...       ...  ...
188       1  0.243492    ”
188       2  0.073460    ”
188       3  0.194996    ”
188       4  0.272943    ”
188       5  0.215111    ”

[1478 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 4, 2])

In [66]:
ps[22]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.041109  0.074038       1        1  33.365927
2      0.007070  0.032714       2        1  19.936007
3     -0.086688 -0.059960       3        1  19.336478
1      0.055240 -0.066248       4        1  17.957511
4      0.065487  0.019456       5        1   9.404077, topic_info=           Term          Freq         Total Category  logprob  loglift
189         gun  16658.000000  16658.000000  Default  30.0000  30.0000
680       woman  11529.000000  11529.000000  Default  29.0000  29.0000
948   protester   8215.000000   8215.000000  Default  28.0000  28.0000
6001      trump  12850.000000  12850.000000  Default  27.0000  27.0000
1368     sexual   3904.000000   3904.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
90          one   1259.964478  15505.649469   Topic5  -5.8059  -0.1461
123       state   1248.112276  15365.253749   Topic5  -5.8153  -0.1464
154        also   1168.195001  13027.398946   Topic5  -5.8815  -0.0476
149   according   1102.487425  10088.866096   Topic5  -5.9394   0.1502
379         law   1079.797637  10118.522729   Topic5  -5.9602   0.1264

[492 rows x 6 columns], token_table=       Topic      Freq Term
term                       
1          1  0.252296   's
1          2  0.220611   's
1          3  0.082914   's
1          4  0.150578   's
1          5  0.293605   's
...      ...       ...  ...
10505      1  0.364775    ”
10505      2  0.246999    ”
10505      3  0.228828    ”
10505      4  0.102944    ”
10505      5  0.056458    ”

[1304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2, 5])

In [67]:
ps[23]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.017740  0.077396       1        1  32.000618
1      0.119327  0.006074       2        1  19.908455
4     -0.043035  0.055007       3        1  18.353195
2     -0.094429 -0.061267       4        1  16.032512
0      0.035876 -0.077210       5        1  13.705221, topic_info=               Term          Freq         Total Category  logprob  loglift
3431        ukraine  20965.000000  20965.000000  Default  30.0000  30.0000
692          russia  23788.000000  23788.000000  Default  29.0000  29.0000
1649        russian  20917.000000  20917.000000  Default  28.0000  28.0000
2783          syria  17775.000000  17775.000000  Default  27.0000  27.0000
7290          trump  34940.000000  34940.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
323   international   2640.936902  11052.284381   Topic5  -6.1331   0.5559
15             also   3168.281435  26425.491368   Topic5  -5.9511  -0.1337
689           right   2608.800060  12206.387081   Topic5  -6.1454   0.4443
98            group   2640.818163  14523.976592   Topic5  -6.1332   0.2827
1                's   2640.653232  21586.603376   Topic5  -6.1332  -0.1137

[521 rows x 6 columns], token_table=       Topic      Freq Term
term                       
1          1  0.397283   's
1          2  0.371156   's
1          3  0.062261   's
1          4  0.046974   's
1          5  0.122344   's
...      ...       ...  ...
16931      1  0.350413    ”
16931      2  0.189968    ”
16931      3  0.204224    ”
16931      4  0.149744    ”
16931      5  0.105652    ”

[1571 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 3, 1])

### Train NyPost LDA

In [84]:
#import nypost data
ny_csvs = [x for x in os.listdir('../data/nypost/') if x.endswith('.csv')]
fns = [os.path.splitext(os.path.basename(x))[0] for x in ny_csvs]
ny_topics=[]
for i in fns:
    ny_topics.append(i.split('_')[1])
ny_d = {}
for i in range(len(fns)):
    ny_d[ny_topics[i]] = pd.read_csv(os.path.join('./nypost',ny_csvs[i]))

In [75]:
#train all topic models for nypost, commented out because it takes a long time to run
# for i in ny_topics:
#     train_lda(ny_d[i], i+'nypost', 5, 10)
#     print(i+'done')

attackdone
Bidendone
blackdone
Chinadone
conflictdone
crimedone
democraticdone
fairdone
gundone
immigrationdone
invasiondone
LGBTdone
policedone
protestdone
refuseddone
republicdone
Russiadone
terrordone
Trumpdone


KeyError: 'UK'

In [87]:
# retrain the following model because error before
#train_lda(ny_d['UK'], 'UK'+'nypost', 5, 10)

In [88]:
#train_lda(ny_d['ukraine'], 'ukraine'+'nypost', 5, 10)

In [89]:
#train_lda(ny_d['US'], 'US'+'nypost', 5, 10)

In [90]:
#train_lda(ny_d['violence'], 'violence'+'nypost', 5, 10)

In [91]:
#train_lda(ny_d['war'], 'war'+'nypost', 5, 10)

In [ ]:
# get pyLDAvis visualizations for all topic models under nypost
ny_topics=[]
ps_ny = []
for i in fns:
    ny_topics.append(i.split('_')[1])
for i in cnn_topics:
    model = models.ldamodel.LdaModel.load('{}nypost.model'.format(i))
    topics = model.print_topics(num_words=20)
    for topic in topics:
        print(topic)
    lemmas = d[i]['text'].apply(get_lemmas)
    dictionary = corpora.Dictionary.load('{}nypost.model.id2word'.format(i))
    corpus = [dictionary.doc2bow(text) for text in lemmas]
    pyLDAvis.enable_notebook()
    p = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    ps_ny.append(p)
    print(i+'done')

(0, '0.013*"new" + 0.009*"—" + 0.009*"city" + 0.008*"said" + 0.007*"york" + 0.007*"state" + 0.007*"crime" + 0.006*"adam" + 0.006*"hochul" + 0.005*"year" + 0.005*"people" + 0.005*"mayor" + 0.004*"law" + 0.003*"would" + 0.003*"one" + 0.003*"democrat" + 0.003*"also" + 0.003*"republican" + 0.003*"like" + 0.003*"zeldin"')
(1, '0.014*"said" + 0.007*"—" + 0.005*"one" + 0.004*"told" + 0.004*"time" + 0.004*"year" + 0.003*"like" + 0.003*"people" + 0.003*"new" + 0.003*"police" + 0.003*"two" + 0.003*"crime" + 0.003*"family" + 0.003*"also" + 0.003*"post" + 0.003*"home" + 0.003*"‘" + 0.002*"would" + 0.002*"get" + 0.002*"man"')
(2, '0.019*"said" + 0.012*"police" + 0.006*"—" + 0.006*"cop" + 0.005*"officer" + 0.005*"crime" + 0.005*"victim" + 0.005*"year" + 0.005*"nypd" + 0.004*"court" + 0.004*"case" + 0.004*"two" + 0.004*"according" + 0.004*"shooting" + 0.004*"gun" + 0.004*"new" + 0.004*"woman" + 0.004*"one" + 0.004*"man" + 0.004*"told"')
(3, '0.016*"said" + 0.007*"police" + 0.005*"told" + 0.005*"—" + 

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


attackdone
(0, '0.008*"—" + 0.008*"said" + 0.006*"year" + 0.006*"tax" + 0.006*"bill" + 0.006*"new" + 0.005*"would" + 0.005*"inflation" + 0.004*"state" + 0.004*"billion" + 0.004*"price" + 0.004*"biden" + 0.004*"million" + 0.004*"also" + 0.004*"company" + 0.003*"federal" + 0.003*"last" + 0.003*"spending" + 0.003*"cost" + 0.003*"american"')
(1, '0.011*"said" + 0.010*"border" + 0.009*"migrant" + 0.008*"biden" + 0.007*"—" + 0.007*"u" + 0.005*"president" + 0.005*"state" + 0.005*"new" + 0.004*"people" + 0.004*"city" + 0.004*"administration" + 0.004*"percent" + 0.004*"texas" + 0.004*"cuomo" + 0.003*"american" + 0.003*"year" + 0.003*"one" + 0.003*"last" + 0.003*"country"')
(2, '0.017*"biden" + 0.012*"president" + 0.010*"trump" + 0.009*"said" + 0.008*"house" + 0.007*"—" + 0.007*"republican" + 0.006*"democrat" + 0.005*"would" + 0.005*"former" + 0.004*"election" + 0.004*"senate" + 0.004*"white" + 0.003*"vote" + 0.003*"time" + 0.003*"one" + 0.003*"american" + 0.003*"year" + 0.003*"image" + 0.003*"j

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


Bidendone
(0, '0.014*"said" + 0.007*"—" + 0.006*"court" + 0.005*"police" + 0.005*"case" + 0.004*"one" + 0.004*"year" + 0.004*"told" + 0.004*"judge" + 0.003*"attorney" + 0.003*"time" + 0.003*"trial" + 0.003*"also" + 0.003*"new" + 0.003*"two" + 0.003*"would" + 0.003*"according" + 0.003*"officer" + 0.003*"fair" + 0.002*"lawyer"')
(1, '0.008*"get" + 0.008*"bet" + 0.008*"new" + 0.006*"one" + 0.005*"—" + 0.005*"best" + 0.005*"bonus" + 0.004*"first" + 0.004*"amazon" + 0.004*"betting" + 0.004*"sport" + 0.003*"game" + 0.003*"available" + 0.003*"make" + 0.003*"giant" + 0.003*"look" + 0.003*"full" + 0.003*"21+" + 0.003*"also" + 0.003*"apply"')
(2, '0.009*"—" + 0.009*"said" + 0.005*"like" + 0.005*"new" + 0.005*"one" + 0.005*"year" + 0.004*"time" + 0.004*"people" + 0.003*"also" + 0.003*"told" + 0.003*"‘" + 0.003*"show" + 0.003*"say" + 0.003*"would" + 0.003*"get" + 0.003*"first" + 0.003*"life" + 0.003*"image" + 0.002*"day" + 0.002*"know"')
(3, '0.010*"said" + 0.008*"—" + 0.006*"new" + 0.006*"state" 

...

conflictdone
(0, '0.012*"said" + 0.008*"—" + 0.005*"one" + 0.005*"new" + 0.004*"home" + 0.004*"year" + 0.004*"like" + 0.003*"time" + 0.003*"two" + 0.003*"police" + 0.003*"say" + 0.003*"woman" + 0.003*"get" + 0.003*"also" + 0.003*"people" + 0.003*"told" + 0.003*"would" + 0.002*"invasion" + 0.002*"day" + 0.002*"first"')
(1, '0.008*"said" + 0.006*"—" + 0.006*"war" + 0.005*"iraq" + 0.005*"u" + 0.005*"president" + 0.004*"would" + 0.004*"state" + 0.004*"one" + 0.004*"american" + 0.004*"year" + 0.004*"obama" + 0.003*"north" + 0.003*"u.s." + 0.003*"military" + 0.003*"people" + 0.003*"korea" + 0.003*"world" + 0.003*"also" + 0.003*"time"')
(2, '0.017*"ukraine" + 0.015*"russian" + 0.015*"said" + 0.013*"russia" + 0.008*"putin" + 0.008*"ukrainian" + 0.008*"president" + 0.007*"u" + 0.005*"—" + 0.005*"war" + 0.005*"biden" + 0.005*"military" + 0.005*"invasion" + 0.005*"force" + 0.005*"country" + 0.004*"would" + 0.004*"people" + 0.003*"also" + 0.003*"official" + 0.003*"troop"')
(3, '0.010*"said" + 0.00

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


crimedone
(0, '0.012*"said" + 0.007*"people" + 0.007*"—" + 0.006*"right" + 0.005*"gay" + 0.005*"woman" + 0.005*"transgender" + 0.005*"say" + 0.005*"lgbt" + 0.004*"law" + 0.004*"year" + 0.004*"also" + 0.004*"state" + 0.003*"one" + 0.003*"gender" + 0.003*"would" + 0.003*"court" + 0.003*"group" + 0.003*"new" + 0.003*"child"')
(1, '0.014*"said" + 0.008*"new" + 0.007*"—" + 0.005*"gay" + 0.005*"state" + 0.005*"city" + 0.004*"lgbt" + 0.004*"people" + 0.004*"york" + 0.004*"year" + 0.004*"would" + 0.004*"one" + 0.004*"community" + 0.003*"right" + 0.003*"cuomo" + 0.003*"also" + 0.003*"time" + 0.003*"group" + 0.003*"marriage" + 0.003*"told"')
(2, '0.010*"said" + 0.009*"—" + 0.009*"gay" + 0.005*"people" + 0.004*"one" + 0.004*"pride" + 0.004*"parade" + 0.004*"lgbt" + 0.004*"year" + 0.004*"community" + 0.004*"city" + 0.004*"new" + 0.004*"police" + 0.003*"also" + 0.003*"day" + 0.003*"trump" + 0.003*"time" + 0.003*"march" + 0.003*"would" + 0.003*"like"')
(3, '0.008*"—" + 0.005*"said" + 0.005*"trump" +

C:\Users\jessi\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


democraticdone
(0, '0.026*"said" + 0.019*"police" + 0.009*"cop" + 0.007*"man" + 0.006*"officer" + 0.006*"victim" + 0.005*"two" + 0.005*"suspect" + 0.005*"according" + 0.005*"—" + 0.005*"shot" + 0.005*"street" + 0.005*"one" + 0.004*"car" + 0.004*"around" + 0.004*"woman" + 0.004*"shooting" + 0.004*"advertisement" + 0.004*"hospital" + 0.004*"nypd"')
(1, '0.008*"said" + 0.006*"—" + 0.004*"police" + 0.004*"told" + 0.004*"one" + 0.004*"family" + 0.004*"time" + 0.004*"also" + 0.003*"year" + 0.003*"advertisement" + 0.003*"game" + 0.003*"new" + 0.003*"image" + 0.003*"two" + 0.003*"friend" + 0.003*"getty" + 0.003*"home" + 0.003*"star" + 0.003*"fan" + 0.003*"day"')
(2, '0.007*"said" + 0.006*"police" + 0.005*"—" + 0.004*"one" + 0.004*"told" + 0.003*"video" + 0.003*"new" + 0.003*"time" + 0.003*"woman" + 0.003*"charge" + 0.003*"according" + 0.003*"year" + 0.003*"also" + 0.003*"two" + 0.003*"allegedly" + 0.002*"family" + 0.002*"court" + 0.002*"post" + 0.002*"officer" + 0.002*"incident"')
(3, '0.019*"

In [ ]:
# display pyLDAvis visualizations for topic models under nypost
ps_ny[0]

In [ ]:
ps_ny[1]

In [ ]:
# war
ps[23]

In [ ]:
ps_ny[23]

In [ ]:
# nypost terror
ps_ny[17]

In [ ]:
# CNN terror
ps[17]

In [ ]:
#ukraine CNN
ps[19]

In [ ]:
# ukraine nypost
ps_ny[19]

In [ ]:
#violence CNN
ps[22]

In [ ]:
#violence nypost
ps_ny[22]

In [ ]:
# gun CNN 
ps[8]

In [ ]:
ps_ny[8]

In [ ]:
# republic CNN
ps[15]

In [ ]:
# republic nypost
ps_ny[15]

In [ ]:
# democrat CNN
ps[6]

In [ ]:
# democrat nypost
ps_ny[6]